# Trabajo Práctico 2: APIs y Web Scraping
## Alumnos: Manuel Díaz de la Fuente, Diego Fernández Mejide y Sofía Kastika
#### 
El objetivo del presente trabajo es la utilización de APIs y de Web Scraping. Para ello, en primer lugar se usará la API de Mercado Libre para un análisis exploratorio de precios de los productos y de items vendidos por ciertos productores. En segundo lugar, se utilizará la técnica de Web Scraping para tomar los links de las noticias de La Nación vigentes al momento de la realización del trabajo. 

### Parte A 

### Parte B
El objetivo de esta parte del trabajo es utilizar Web Scraping para armar un dataframe con los los títulos y los links de las noticias expuestas en La Nación el 9/10/2024. 

ACA PONER LA CAPTURA DE PANTALLA DE LA NACIÓN. 

### Importamos librerías

In [26]:
import requests # html requestor
from bs4 import BeautifulSoup # html parser 
import pandas as pd # Librería para manipular dataframes

### Accediendo al contenido de la página de La Nación 

In [70]:
url = "https://www.lanacion.com.ar/" # Definimos URL
r = requests.get(url) # Traemos a la computadora el contenido html y lo almacenamos en 'r' 
contenido = r.content # Accedemos al contenido de r con .content (en formato bytes)
soup = BeautifulSoup(contenido, "html.parser") # Que se procese el contenido con el parser

### Dentro de la sopa, buscamos los elementos que contienen la información que necesitamos: 

### Títulos de noticias

In [73]:
h2_elements = soup.find_all('h2') # Que encuentre en la sopa todos los elementos h2 
titulos = [] # Creamos una lista vacía para luego guardar los títulos ahí

# Armamos un for loop: 
for i in h2_elements: # Para cada elemento en h2_elements
    h2_text = i.text.strip() # Guardar al elemento como texto y sacando los espacios necesarios con .strip()
    titulos.append({  
        'titular': h2_text
    }) # Armamos un diccionario para cada elemento i en donde la key es 'titular' y el valor es 'h2_text' y los agregamos a la lista 'titulos'

### Creamos un dataframe a partir de la lista de títulos 

In [76]:
titulos_df = pd.DataFrame(titulos) # Con esto tenemos un archivo con todos los titulares de la nación al momento 
titulos_df # Vemos los titulos

,titular
0,"El oficialismo, en alianza con Pro, logró 85 v..."
1,Aliados y sorpresas. Así votó cada diputado la...
2,¿Se viene otro asado? Quiénes fueron los “85 h...
3,Ley de financiamiento. Los gremios repudiaron ...
4,"""Es un gladiador"". La salud de Jorge Lanata: f..."
...,...
123,Noches de cine y series. Ideas para ubicar la ...
124,Descanso. A quiénes les corresponde el feriado...
125,Fechas de cobro. Así es el calendario de pagos...
126,¿Cuánto ganan? Confirman nuevo aumento de suel...


## Links de noticias 
En el caso de las noticias de la nación, las noticias tienen el tag 'a', y son de la clase 'link ln-link flex flex-column --unstyled' 

NO SE SI ES EN EL CASO DE LAS NOTICIAS DE LA NACIÓN O SI ES SIEMPRE ASÍ

In [89]:
a_elements = soup.find_all('a', class_='link ln-link flex flex-column --unstyled') # Que encuentre en la sopa todos los elementos a con la clase mencionada. 

# Comentario: utilizamos class_ porque 'class' es una palabra protegida

links = [] # Creamos una lista vacía para luego guardar los links ahí
base_url = 'https://www.lanacion.com.ar'
# Armamos un for loop para extraer el href para cada link
for i in a_elements:
    href = i.get('href')  # Obtenemos el atributo href
    full_url = base_url + href 
    links.append({
        'link': full_url,
        })  # Armamos un diccionario para cada link y lo agrego a la lista 'links'


### Creamos un dataframe a partir de la lista de links  

In [92]:
links_df = pd.DataFrame(links) # Con esto tenemos un archivo con todos los links de los titulos la nación al momento 
links_df # vemos los links

,link
0,https://www.lanacion.com.ar/politica/veto-de-m...
1,https://www.lanacion.com.ar/politica/aliados-y...
2,https://www.lanacion.com.ar/politica/universid...
3,https://www.lanacion.com.ar/sociedad/ley-de-fi...
4,https://www.lanacion.com.ar/espectaculos/perso...
...,...
123,https://www.lanacion.com.ar/revista-living/bue...
124,https://www.lanacion.com.ar/feriados/fin-de-se...
125,https://www.lanacion.com.ar/economia/calendari...
126,https://www.lanacion.com.ar/economia/confirman...


### Ahora, vamos a unir los dataframes de los títulos de las noticias y sus links para que nos quede toda la información en un mismo dataframe

In [87]:
# Concatenamoss los dataframes por columna
result_df = pd.concat([titulos_df, links_df], axis=1)

# Cambiamos los títulos 
titulos_df.columns = ['Noticia']
links_df.columns = ['Link']

# Exportamos a excel 
result_df.to_excel("C:/Users/sofia/Desktop/Maestría/Tercer trimestre/Machine Learning/TPs (mio)/TP 2/hola.xlsx", index=False, engine='openpyxl')